In [1]:
spark

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.0.7:4041
SparkContext available as 'sc' (version = 3.1.2, master = local[*], app id = local-1643873322399)
SparkSession available as 'spark'


res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@58508a03


앞 장에서는 RDD를 다루기 위한 기초적인 내용과 RDD를 어떻게 생성하는지 그리고 어떤 경우에 RDD를 사용해야 하는지 알아봤음<br/>
RDD 데이터를 변환하기 위한 map, filter, reduce 함수와 사용자 정의 함수를 만드는 방법도 함께 알아봤음<br/>
이 장에서는 키-값 형태의 RDD를 중심으로 RDD 고급 연산에 대해 알아보겠음<br/>
또한 사용자 정의 파티션과 같은 고급 주제에 대해서도 알아보겠음<br/>

사용자 정의 파티션 함수를 사용하면 클러스터에 데이터가 배치되는 방식을 정확히 제어할 수 있으며 개별 파티션을 다룰 수도 있음<br/>
13장에서 다룰 핵심 주제는 다음과 같음<br/>
* 집계와 키-값 형태의 RDD
* 사용자 정의 파티셔닝
* RDD 조인
<br/>

12장에서 사용했던 데이터셋을 이 장에서도 계속 사용함<br/>

In [2]:
val myCollection = "Spark The Definitive Guide : Big Data Processing Made Simple".split(" ")
val words = spark.sparkContext.parallelize(myCollection, 2)

myCollection: Array[String] = Array(Spark, The, Definitive, Guide, :, Big, Data, Processing, Made, Simple)
words: org.apache.spark.rdd.RDD[String] = ParallelCollectionRDD[0] at parallelize at <console>:25


In [8]:
words.collect()

res5: Array[String] = Array(Spark, The, Definitive, Guide, :, Big, Data, Processing, Made, Simple)


# 13.1 키-값 형태의 기초(키-값 형태의 RDD)

RDD에는 데이터를 키-값 형태로 다룰 수 있는 다양한 메서드가 있음<br/>
이러한 메서드는 <연산명>ByKey 형태의 이름을 가짐<br/>
메서드 이름에 ByKey가 있다면 PairRDD 타입만 사용할 수 있음<br/>
PairRDD 타입을 만드는 가장 쉬운 방법은 RDD에 맵 연산을 수행해 키-값 구조로 만드는 것임<br/>
즉, RDD 레코드에 두 개의 값이 존재함<br/>

In [3]:
words.map(word => (word.toLowerCase, 1))

res1: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[1] at map at <console>:28


## 13.1.1 keyBy

위에서 키를 만드는 간단한 방법에 대해 알아봤음<br/>
현재 값으로부터 키를 생성하는 keyBy 함수를 사용해 동일한 결과를 얻을 수 있음<br/>
다음 예제에서는 단어의 첫 번째 문자를 키로 만들어 RDD를 생성함<br/>
이때 스파크는 원본 단어를 생성된 RDD의 값으로 유지함<br/>

In [4]:
val keyword = words.keyBy(word => word.toLowerCase.toSeq(0).toString)

keyword: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[2] at keyBy at <console>:26


In [7]:
keyword.collect()

res4: Array[(String, String)] = Array((s,Spark), (t,The), (d,Definitive), (g,Guide), (:,:), (b,Big), (d,Data), (p,Processing), (m,Made), (s,Simple))


## 13.1.2 값 매핑하기

생성된 키-값 셋을 사용해 데이터를 다뤄보겠음<br/>
만약 튜플 형태의 데이터를 사용한다면 스파크는 튜플의 첫 번째 요소를 키로, 두 번째 요소를 값으로 추정함<br/>
튜플 형태의 데이터에서 키를 제외하고 값만 추출할 수 있음<br/>
직접 구현할 수도 있지만, 다음 예제와 같이 mapValues 메서드를 사용하면 값 수정 시 발생할 수 있는 오류를 미리 방지할 수 있음<br/>

In [5]:
keyword.mapValues(word => word.toUpperCase).collect()

res2: Array[(String, String)] = Array((s,SPARK), (t,THE), (d,DEFINITIVE), (g,GUIDE), (:,:), (b,BIG), (d,DATA), (p,PROCESSING), (m,MADE), (s,SIMPLE))


flapMap 함수를 사용해 반환되는 결과의 각 로우가 문자를 나타내도록 확장할 수 있음<br/>
마음 예제의 출력 결과는 생략되어 있지만, 단어의 첫 글자를 키로, 단어의 각 문자를 값으로 하는 배열이 생성됨<br/>

In [6]:
keyword.flatMapValues(word => word.toUpperCase).collect()

res3: Array[(String, Char)] = Array((s,S), (s,P), (s,A), (s,R), (s,K), (t,T), (t,H), (t,E), (d,D), (d,E), (d,F), (d,I), (d,N), (d,I), (d,T), (d,I), (d,V), (d,E), (g,G), (g,U), (g,I), (g,D), (g,E), (:,:), (b,B), (b,I), (b,G), (d,D), (d,A), (d,T), (d,A), (p,P), (p,R), (p,O), (p,C), (p,E), (p,S), (p,S), (p,I), (p,N), (p,G), (m,M), (m,A), (m,D), (m,E), (s,S), (s,I), (s,M), (s,P), (s,L), (s,E))


## 13.1.3 키와 값 추출하기

키-값 형태의 데이터를 가지고 있다면 다음 메서드를 사용해 키나 값 전체를 추출할 수 있음<br/>

In [10]:
keyword.keys.collect()

res7: Array[String] = Array(s, t, d, g, :, b, d, p, m, s)


In [11]:
keyword.values.collect()

res8: Array[String] = Array(Spark, The, Definitive, Guide, :, Big, Data, Processing, Made, Simple)


## 13.1.4 lookup

RDD를 사용해서 할 수 있는 흥미로운 작업 중 하나는 특정 키에 관한 결과를 찾는 것임<br/>
그러나 각 입력에 대해 오직 하나의 키만 찾을 수 있도록 강제하는 메커니즘은 없음<br/>
예를 들어 lookup 함수의 인수로 's'를 입력하면 키가 's'인 'Spark'와 'Simple'을 반환함<br/>

In [12]:
keyword.lookup("s")

res9: Seq[String] = WrappedArray(Spark, Simple)


## 13.1.5 sampleByKey

근사치나 정확도를 이용해 키를 기반으로 RDD 샘플을 생성할 수 있음<br/>
두 작업 모두 특정 키를 부분 샘플링할 수 있으며 선택에 따라 비복원 추출을 사용할 수도 있음<br/>
다음 예제의 sampleByKey 메서드는 RDD를 한 번만 처리하면서 간단한 무작위 샘플링을 사용하기 때문에 모든 키값에 대한 math.ceil(numItems * samplingRate) 값의 총합과 거의 동일한 크기의 샘플을 생성함<br/>

In [13]:
val distinctChars = words.flatMap(word => word.toLowerCase.toSeq).distinct.collect()

distinctChars: Array[Char] = Array(d, p, t, b, h, n, f, v, :, r, l, s, e, a, i, k, u, o, g, m, c)


In [15]:
import scala.util.Random

val sampleMap = distinctChars.map(c => (c, new Random().nextDouble())).toMap // nextDouble은 [0.0, 1.0) 범위의 부동소수점 난수

import scala.util.Random
sampleMap: scala.collection.immutable.Map[Char,Double] = Map(e -> 0.7007467039283193, s -> 0.7428428231669688, n -> 0.23144560173878903, t -> 0.025427929298935337, u -> 0.982003390405394, f -> 0.2011075841669001, a -> 0.9162384543460105, m -> 0.016693383960537878, i -> 0.7264537944154682, v -> 0.8314395491847529, b -> 0.9718207084038761, g -> 0.6767402693985604, l -> 0.9276296964178412, p -> 0.21239627506536451, c -> 0.2257046025041367, h -> 0.6532178174839449, r -> 0.8910142364744786, : -> 0.665860782769436, k -> 0.3999062167582884, o -> 0.7584260553061055, d -> 0.5456457758753983)


In [21]:
words.map(word => (word.toLowerCase.toSeq(0), word))
    .sampleByKey(true, sampleMap, 6L)
    .collect()

res16: Array[(Char, String)] = Array((s,Spark), (d,Definitive), (g,Guide), (:,:))


다음 예제의 sampleByKeyExact 메서드는 99.99% 신뢰도를 가진 모든 키값에 대해 RDD를 추가로 처리함<br/>
그리고 *math.ceil(numItems * samplingRate)의 합과 완전히 동일한 크기의 샘플 데이터를 생성*하므로 sampleByKey 함수와는 다름<br/>
비복원 추출을 사용한다면 샘플 크기를 보장하기 위해 RDD를 한 번 더 통과해야 하며, 복원 추출을 사용한다면 RDD를 두 번 더 통과해야 함<br/>

In [22]:
words.map(word => (word.toLowerCase.toSeq(0), word))
    .sampleByKeyExact(true, sampleMap, 6L)
    .collect()

res17: Array[(Char, String)] = Array((s,Spark), (t,The), (d,Definitive), (g,Guide), (:,:), (b,Big), (d,Data), (p,Processing), (m,Made), (s,Simple))


# 13.2 집계

사용하는 메서드에 따라 일반 RDD나 PairRDD를 사용해 집계를 수행할 수 있음<br/>
words 데이터셋을 이용해 자세히 알아보겠음<br/>

In [23]:
val chars = words.flatMap(word => word.toLowerCase.toSeq)
val KVcharacters = chars.map(letter => (letter, 1))

def maxFunc(left:Int, right:Int) = math.max(left, right)
def addFunc(left:Int, right:Int) = left + right

val nums = sc.parallelize(1 to 30, 5)

chars: org.apache.spark.rdd.RDD[Char] = MapPartitionsRDD[43] at flatMap at <console>:30
KVcharacters: org.apache.spark.rdd.RDD[(Char, Int)] = MapPartitionsRDD[44] at map at <console>:31
maxFunc: (left: Int, right: Int)Int
addFunc: (left: Int, right: Int)Int
nums: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[45] at parallelize at <console>:36


In [29]:
KVcharacters.collect()

res23: Array[(Char, Int)] = Array((s,1), (p,1), (a,1), (r,1), (k,1), (t,1), (h,1), (e,1), (d,1), (e,1), (f,1), (i,1), (n,1), (i,1), (t,1), (i,1), (v,1), (e,1), (g,1), (u,1), (i,1), (d,1), (e,1), (:,1), (b,1), (i,1), (g,1), (d,1), (a,1), (t,1), (a,1), (p,1), (r,1), (o,1), (c,1), (e,1), (s,1), (s,1), (i,1), (n,1), (g,1), (m,1), (a,1), (d,1), (e,1), (s,1), (i,1), (m,1), (p,1), (l,1), (e,1))


위 코드 예제를 수행한 다음 키별 아이템 수를 구하기 위해 countByKey 메서드를 사용함<br/>

## 13.2.1 countByKey

countByKey 메서드는 각 키의 아이템 수를 구하고 로컬 맵으로 결과를 수집함<br/>
스칼라나 자바를 사용한다면 countByKey 메서드에 제한 시간(timeout)과 신뢰도를 인수로 지정해 근사치를 구할 수 있음<br/>

In [24]:
val timeout = 1000L // 밀리세컨드 단위
val confidence = 0.95

KVcharacters.countByKey()

timeout: Long = 1000
confidence: Double = 0.95
res18: scala.collection.Map[Char,Long] = Map(e -> 7, s -> 4, n -> 2, t -> 3, u -> 1, f -> 1, a -> 4, m -> 2, i -> 7, v -> 1, b -> 1, g -> 3, l -> 1, p -> 3, c -> 1, h -> 1, r -> 2, : -> 1, k -> 1, o -> 1, d -> 4)


In [25]:
KVcharacters.countByKeyApprox(timeout, confidence)

res19: org.apache.spark.partial.PartialResult[scala.collection.Map[Char,org.apache.spark.partial.BoundedDouble]] = (final: Map(e -> [7.000, 7.000], s -> [4.000, 4.000], n -> [2.000, 2.000], t -> [3.000, 3.000], u -> [1.000, 1.000], f -> [1.000, 1.000], a -> [4.000, 4.000], m -> [2.000, 2.000], i -> [7.000, 7.000], v -> [1.000, 1.000], b -> [1.000, 1.000], g -> [3.000, 3.000], l -> [1.000, 1.000], p -> [3.000, 3.000], c -> [1.000, 1.000], h -> [1.000, 1.000], r -> [2.000, 2.000], : -> [1.000, 1.000], k -> [1.000, 1.000], o -> [1.000, 1.000], d -> [4.000, 4.000]))


## 13.2.2 집계 연산 구현 방식 이해하기

키-값 형태의 PairRDD를 생성하는 몇 가지 방법이 있음<br/>
이때 구현 방식은 job의 안정성을 위해 매우 중요함<br/>
이를 설명하기 위해 groupBy와 reduce 함수를 비교해보겠음<br/>
groupBy와 reduce 함수 모두 동일한 기본 원칙이 적용되므로 키를 기준으로 비교함<br/>

### groupByKey

In [27]:
KVcharacters.groupByKey().collect()

res21: Array[(Char, Iterable[Int])] = Array((d,CompactBuffer(1, 1, 1, 1)), (p,CompactBuffer(1, 1, 1)), (t,CompactBuffer(1, 1, 1)), (b,CompactBuffer(1)), (h,CompactBuffer(1)), (n,CompactBuffer(1, 1)), (f,CompactBuffer(1)), (v,CompactBuffer(1)), (:,CompactBuffer(1)), (r,CompactBuffer(1, 1)), (l,CompactBuffer(1)), (s,CompactBuffer(1, 1, 1, 1)), (e,CompactBuffer(1, 1, 1, 1, 1, 1, 1)), (a,CompactBuffer(1, 1, 1, 1)), (k,CompactBuffer(1)), (i,CompactBuffer(1, 1, 1, 1, 1, 1, 1)), (u,CompactBuffer(1)), (o,CompactBuffer(1)), (g,CompactBuffer(1, 1, 1)), (m,CompactBuffer(1, 1)), (c,CompactBuffer(1)))


API 문서 내용대로라면 각 키의 총 레코드 수를 구하는 경우 groupByKey의 결과로 만들어진 그룹에 map 연산을 수행하는 방식이 가장 좋아보임<br/>

In [26]:
KVcharacters.groupByKey().map(row => (row._1, row._2.reduce(addFunc))).collect()

org.apache.spark.SparkException:  Job aborted due to stage failure: Task 0 in stage 35.0 failed 1 times, most recent failure: Lost task 0.0 in stage 35.0 (TID 70) (192.168.0.7 executor driver): java.lang.NullPointerException

*의문사항*<br/>
책에서는 위의 코드가 에러가 난다고 되어 있지 않은데 왜 에러가 날까? <br/>
당장 생각하기로는 java.lang.NullPointerException 에러 문구를 볼 때 row._2.reduce(addFunc) 부분 계산을 할 때 (l,CompactBuffer(1)) 같은 케이스에서는 하나밖에 없는 것에 대해 left + right 연산을 수행해야 하는데 여기서 right가 null로 처리돼서 그런 게 아닌가 싶음<br/>

(이어서)<br/>
하지만 잘못된 접근 방법인 경우가 많음<br/>
여기서 가장 큰 문제는 모든 executor에서 함수를 적용하기 전에 해당 키와 관련된 **모든 값**을 메모리로 읽어들여야 한다는 것임<br/>
그 이유는 심각하게 치우쳐진 키가 있다면 일부 파티션이 엄청난 양의 값을 가질 수 있으므로 OutOfMemoryError가 발생하기 때문임<br/>
예제에서 사용하는 데이터셋으로는 OutOfMemoryError가 발생하지 않겠지만, 대규모 분산 환경에서는 심각한 문제로 이어질 수 있음<br/>

그러므로 각 키에 대한 값의 크기가 일정하고 executor에 할당된 메모리에서 처리 가능할 정도의 크기라면 groupByKey 메서드를 사용하되 groupByKey 메서드를 사용할 때 어떤 일이 발생하는지 정확하게 알아야 함<br/>
또 다른 해결 방안인 reduceByKey 메서드를 알아보겠음<br/>

### reduceByKey

지금은 단순하게 개수를 구하고 있으므로 flatMap을 동일하게 수행한 다음 map 연산을 사용해 각 문자의 인스턴스를 1로 매핑함 (KVcharacters)<br/>
그런 다음 결과값을 배열에 모을 수 있도록 합계 함수와 함께 reduceByKey 메서드를 수행함<br/>
이러한 구현 방식은 각 파티션에서 reduce 작업을 수행하기 때문에 훨씬 안정적이며 모든 값을 메모리에 유지하지 않아도 됨<br/>
또한 최종 reduce 과정을 제외한 모든 작업은 개별 워커에서 처리하기 때문에 연산 중에 셔플이 발생하지 않음<br/>
그러므로 이러한 방식을 사용하면 안정성뿐만 아니라 연산 수행 속도가 크게 향상됨<br/>

In [28]:
KVcharacters.reduceByKey(addFunc).collect()

res22: Array[(Char, Int)] = Array((d,4), (p,3), (t,3), (b,1), (h,1), (n,2), (f,1), (v,1), (:,1), (r,2), (l,1), (s,4), (e,7), (a,4), (i,7), (k,1), (u,1), (o,1), (g,3), (m,2), (c,1))


reduceByKey 메서드는 키별 그룹 RDD를 반환함<br/>
그러나 RDD의 개별 요소들은 정렬되어 있지 않음<br/>
따라서 이 메서드는 작업부하를 줄이려는 경우에 적합함 (sorting을 안 하니까)<br/>
반면 결과의 순서가 중요한 경우에는 적합하지 않음<br/>

## 13.2.3 기타 집계 메서드

다양한 고급 집계 함수가 존재하며 사용자 워크로드에 따라 세부 구현 방법에서 차이가 날 수 있음<br/>
최신 버전의 스파크를 사용하면 워크로드의 종류(또는 이와 유사한 명령을 수행하는 경우)에 따른 세부 구현의 차이를 거의 느낄 수 없을 것임<br/>
즉, 구조적 API를 사용하면 훨씬 간단하게 집계를 수행할 수 있으므로 굳이 고급 집계 함수를 사용하지 않아도 됨<br/>
하지만 고급 집계 함수를 사용해 클러스터 노드에서 수행하는 집계를 아주 구체적이고 매우 세밀하게 제어할 수 있음<br/>

### aggregate

aggregate 함수는 null 값이나 집계의 시작값이 필요하며 두 가지 함수를 파라미터로 사용함<br/>
첫 번째 함수는 파티션 내에서 수행되고 두 번째 함수는 모든 파티션에 걸쳐 수행됨<br/>
두 함수 모두 시작값을 사용함<br/>

In [30]:
nums.aggregate(0)(maxFunc, addFunc)

res24: Int = 90


aggregate 함수는 드라이버에서 최종 집계를 수행하므로 성능에 약간 영향이 있음<br/>
예를 들어 executor의 결과가 너무 크면 OutOfMemoryError가 발생해 드라이버가 비정상적으로 종료됨<br/>

*aggregate 함수와 동일한 작업을 수행하지만 다른 처리 과정을 거치는 treeAggregate 함수도 있음*<br/>
**이 함수는 기본적으로 드라이버에서 최종 집계를 수행하기 전에 executor끼리 트리를 형성해 집계 처리의 일부 하위 과정을 'push down' 방식으로 먼저 수행함**<br/>
집계 처리를 여러 단계로 구성하는 것은 드라이버의 메모리를 모두 소비하는 현상을 막는 데 도움이 됨<br/>
이러한 트리 기반의 구현 방식은 작업의 안정성을 높이기 위해 사용하기도 함<br/>

In [31]:
val depth = 3
nums.treeAggregate(0)(maxFunc, addFunc, depth)

depth: Int = 3
res25: Int = 90


aggregate 함수의 내부 동작에 대한 더 자세한 설명은 [이 블로그 글](https://whereami80.tistory.com/105)을 참고할 것

### aggregateByKey

aggregateByKey 함수는 aggregate 함수와 동일하지만 파티션 대신 키를 기준으로 연산을 수행함<br/>
시작값과 함수는 이전 예제와 동일하게 사용함<br/>

In [32]:
KVcharacters.aggregateByKey(0)(addFunc, maxFunc).collect()

res26: Array[(Char, Int)] = Array((d,2), (p,2), (t,2), (b,1), (h,1), (n,1), (f,1), (v,1), (:,1), (r,1), (l,1), (s,3), (e,4), (a,3), (i,4), (k,1), (u,1), (o,1), (g,2), (m,2), (c,1))


### combineByKey

combineByKey 함수는 집계 함수 대신 combiner를 사용함<br/>
이 combiner는 키를 기준으로 연산을 수행하며 파라미터로 사용된 함수에 따라 값을 병합함<br/>
그런 다음 여러 combiner의 결과값을 병합해 결과를 반환함<br/>
사용자 정의 파티셔너를 사용해 출력 파티션 수를 지정할 수도 있음<br/>

In [33]:
val valToCombiner = (value:Int) => List(value)
val mergeValuesFunc = (vals:List[Int], valToAppend:Int) => valToAppend :: vals
val mergeCombinerFunc = (vals1:List[Int], vals2:List[Int]) => vals1 ::: vals2
//함수형 변수를 정의할 수도 있음
val outputPartitions = 6

KVcharacters
    .combineByKey(
        valToCombiner,
        mergeValuesFunc,
        mergeCombinerFunc,
        outputPartitions)
    .collect()

valToCombiner: Int => List[Int] = $Lambda$3197/310586960@52dc297f
mergeValuesFunc: (List[Int], Int) => List[Int] = $Lambda$3198/2046125131@223bb4c7
mergeCombinerFunc: (List[Int], List[Int]) => List[Int] = $Lambda$3199/1684201390@136aa95
outputPartitions: Int = 6
res27: Array[(Char, List[Int])] = Array((f,List(1)), (r,List(1, 1)), (l,List(1)), (s,List(1, 1, 1, 1)), (a,List(1, 1, 1, 1)), (g,List(1, 1, 1)), (m,List(1, 1)), (t,List(1, 1, 1)), (b,List(1)), (h,List(1)), (n,List(1, 1)), (i,List(1, 1, 1, 1, 1, 1, 1)), (u,List(1)), (o,List(1)), (c,List(1)), (d,List(1, 1, 1, 1)), (p,List(1, 1, 1)), (v,List(1)), (:,List(1)), (e,List(1, 1, 1, 1, 1, 1, 1)), (k,List(1)))


### foldByKey

foldByKey 함수는 결합 함수와 항등원인 '제로값'을 이용해 각 키의 값을 병합함<br/>
제로값은 결과에 따라 여러 번 사용될 수 있으나 결과를 변경할 수는 없음 (예: 덧셈에서는 0, 곱셈에서는 1)<br/>

In [34]:
KVcharacters.foldByKey(0)(addFunc).collect()

res28: Array[(Char, Int)] = Array((d,4), (p,3), (t,3), (b,1), (h,1), (n,2), (f,1), (v,1), (:,1), (r,2), (l,1), (s,4), (e,7), (a,4), (i,7), (k,1), (u,1), (o,1), (g,3), (m,2), (c,1))


# 13.3 cogroup

cogroup 함수는 스칼라를 사용하는 경우 최대 3개, 파이썬을 사용하는 경우 최대 2개의 키-값 형태의 RDD를 그룹화할 수 있으며 각 키를 기준으로 값을 결합함<br/>
즉, RDD에 대한 그룹 기반의 조인을 수행함<br/>
cogroup 함수는 출력 파티션 수나 클러스터에 데이터 분산 방식을 정확하게 제어하기 위해 사용자 정의 파티션 함수를 파라미터로 사용할 수 있음<br/>

In [35]:
import scala.util.Random

val distinctChars = words.flatMap(word => word.toLowerCase.toSeq).distinct
val charRDD = distinctChars.map(c => (c, new Random().nextDouble()))
val charRDD2 = distinctChars.map(c => (c, new Random().nextDouble()))
val charRDD3 = distinctChars.map(c => (c, new Random().nextDouble()))

charRDD.cogroup(charRDD2, charRDD3).take(5)

import scala.util.Random
distinctChars: org.apache.spark.rdd.RDD[Char] = MapPartitionsRDD[60] at distinct at <console>:33
charRDD: org.apache.spark.rdd.RDD[(Char, Double)] = MapPartitionsRDD[61] at map at <console>:34
charRDD2: org.apache.spark.rdd.RDD[(Char, Double)] = MapPartitionsRDD[62] at map at <console>:35
charRDD3: org.apache.spark.rdd.RDD[(Char, Double)] = MapPartitionsRDD[63] at map at <console>:36
res29: Array[(Char, (Iterable[Double], Iterable[Double], Iterable[Double]))] = Array((d,(CompactBuffer(0.5142862688562798),CompactBuffer(0.23750928677650407),CompactBuffer(0.68575508076696))), (p,(CompactBuffer(0.02524065526976682),CompactBuffer(0.20622158234605048),CompactBuffer(0.22281333077058318))), (t,(CompactBuffer(0.40757915378422915),CompactBuffer(0.5307295800641897),Compact...


In [36]:
distinctChars.collect()

res30: Array[Char] = Array(d, p, t, b, h, n, f, v, :, r, l, s, e, a, i, k, u, o, g, m, c)


그룹화된 키를 '키'로, 키와 관련된 모든 값을 '값'으로 하는 키-값 형태의 배열을 결과로 반환함<br/>

# 13.4 조인

RDD는 구조적 API에서 알아본 것과 거의 동일한 조인 방식을 가지고 있지만 RDD를 사용하면 사용자가 많은 부분에 관여해야 함<br/>
RDD나 구조적 API의 조인 방식 모두 동일한 기본 형식을 사용함<br/>
조인하려는 두 개의 RDD가 기본적으로 필요함<br/>
때에 따라 출력 파티션 수나 사용자 정의 파티션 함수를 파라미터로 사용함<br/>
파티션 함수와 관련된 내용은 13.5.4절 '사용자 정의 파티셔닝'에서 자세히 알아보겠음<br/>

## 13.4.1 내부 조인

출력 파티션 수를 어떻게 설정하는지 주의 깊게 살펴보자<br/>

In [38]:
val keyedChars = distinctChars.map(c => (c, new Random().nextDouble()))
val outputPartitions = 10

KVcharacters.join(keyedChars).count()

keyedChars: org.apache.spark.rdd.RDD[(Char, Double)] = MapPartitionsRDD[73] at map at <console>:35
outputPartitions: Int = 10
res32: Long = 51


In [39]:
KVcharacters.join(keyedChars, outputPartitions).count()

res33: Long = 51


In [40]:
KVcharacters.join(keyedChars).collect()

res34: Array[(Char, (Int, Double))] = Array((d,(1,0.31855062615921215)), (d,(1,0.31855062615921215)), (d,(1,0.31855062615921215)), (d,(1,0.31855062615921215)), (p,(1,0.9512515408182779)), (p,(1,0.9512515408182779)), (p,(1,0.9512515408182779)), (t,(1,0.7501740150774551)), (t,(1,0.7501740150774551)), (t,(1,0.7501740150774551)), (b,(1,0.9971539720548231)), (h,(1,0.7540189439406084)), (n,(1,0.6111082758907096)), (n,(1,0.6111082758907096)), (f,(1,0.13005600730867883)), (v,(1,0.9634213556490557)), (:,(1,0.4516340554706638)), (r,(1,0.03354323320469721)), (r,(1,0.03354323320469721)), (l,(1,0.8308799090803691)), (s,(1,0.7216844543072045)), (s,(1,0.7216844543072045)), (s,(1,0.7216844543072045)), (s,(1,0.7216844543072045)), (e,(1,0.4574756147023401)), (e,(1,0.4574756147023401)), (e,(1,0.4574756147...


## 13.4.2 zip

마지막 조인 타입은 사실 진짜 조인은 아니지만 두 개의 RDD를 결합하므로 조인이라 볼 수 있음<br/>
zip 함수를 사용해 동일한 길이의 두 개의 RDD를 지퍼 잠그듯이 연결할 수 있으며 PairRDD를 생성함<br/>
두 개의 RDD는 동일한 수의 요소와 동일한 수의 파티션을 가져야 함<br/>

In [41]:
val numRange = sc.parallelize(0 to 9, 2)
words.zip(numRange).collect()

numRange: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[83] at parallelize at <console>:32
res35: Array[(String, Int)] = Array((Spark,0), (The,1), (Definitive,2), (Guide,3), (:,4), (Big,5), (Data,6), (Processing,7), (Made,8), (Simple,9))


# 13.5 파티션 제어하기

RDD를 사용하면 데이터가 클러스터 전체에 물리적으로 정확히 분산되는 방식을 정의할 수 있음<br/>
이러한 기능을 가진 메서드 중 일부는 구조적 API에서 사용했던 메서드와 기본적으로 동일함<br/>
**구조적 API와 가장 큰 차이점은 파티션 함수를 파라미터로 사용할 수 있다는 사실임**<br/>
**파티션 함수는 보통 사용자 지정 Partitioner를 의미**하며 기본 메서드를 살펴볼 때 자세히 알아보겠음<br/>

## 13.5.1 coalesce

**coalesce는 파티션을 재분배할 때 발생하는 데이터 셔플을 방지하기 위해 동일한 워커에 존재하는 파티션을 합치는 메서드임**<br/>
예를 들어 words RDD는 현재 두 개의 파티션으로 구성되어 있음<br/>
coalesce 메서드를 사용해 데이터 셔플링 없이 하나의 파티션으로 합칠 수 있음<br/>

In [42]:
words.coalesce(1).getNumPartitions // 값은 1

res36: Int = 1


## 13.5.2 repartition

repartition 메서드를 사용해 파티션 수를 늘리거나 줄일 수 있지만, 처리 시 노드 간의 셔플이 발생할 수 있음<br/>
파티션 수를 늘리면 맵 타입이나 필터 타입의 연산을 수행할 때 병렬 처리 수준을 높일 수 있음<br/>

In [43]:
words.repartition(10) // 10개의 파티션이 생성됨

res37: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[89] at repartition at <console>:33


## 13.5.3 repartitionAndSortWithinPartitions

이 메서드는 파티션을 재분배할 수 있고, 재분배된 결과 파티션의 정렬 방식을 지정할 수 있음<br/>
파티셔닝과 키 비교 모두 사용자가 지정할 수 있음<br/>

## 13.5.4 사용자 정의 파티셔닝

사용자 정의 파티셔닝(custom partitioning)은 RDD를 사용하는 가장 큰 이유 중 하나임<br/>
**논리적인 대응책을 가지고 있지 않으므로 구조적 API에서는 사용자 정의 파티셔너를 파라미터로 사용할 수 없음**<br/>
사용자 정의 파티셔너는 저수준 API의 세부적인 구현 방식임<br/>
사용자 정의 파티셔닝의 대표적인 예제는 PageRank임<br/>
PageRank는 사용자 정의 파티셔닝을 이용해 클러스터의 데이터 배치 구조를 제어하고 셔플을 회피함<br/>
예제에서는 쇼핑 데이터셋을 고객 ID별로 파티셔닝함<br/>

**사용자 정의 파티셔닝의 유일한 목표는 데이터 치우침(skew) 같은 문제를 피하고자 클러스터 전체에 걸쳐 데이터를 균등하게 분배하는 것임**<br/>

사용자 정의 파티셔너를 사용하려면 구조적 API로 RDD를 얻고 사용자 정의 파티셔너를 적용한 다음 다시 DataFrame이나 Dataset으로 변환해야 함<br/>
*이 방법은 필요시에만 사용자 정의 파티셔닝을 사용할 수 있으므로 구조적 API와 RDD의 장점을 모두 활용할 수 있음*<br/>

사용자 정의 파티셔닝을 사용하려면 Partitioner를 확장한 클래스를 구현해야 함<br/>
문제에 대한 업무 지식을 충분히 가지고 있는 경우에만 사용해야 함<br/>
단일 값이나 다수 값(다수 컬럼)을 파티셔닝해야 한다면 DataFrame API를 사용하는 것이 좋음<br/>

예제를 통해 자세히 알아보겠음<br/>

In [44]:
val df = spark.read.option("header", "true").option("inferSchema", "true")
    .csv("Downloads/Spark-The-Definitive-Guide/data/retail-data/all/")
val rdd = df.coalesce(10).rdd

df: org.apache.spark.sql.DataFrame = [InvoiceNo: string, StockCode: string ... 6 more fields]
rdd: org.apache.spark.rdd.RDD[org.apache.spark.sql.Row] = MapPartitionsRDD[105] at rdd at <console>:31


HashPartitioner와 RangePartitioner는 RDD API에서 사용할 수 있는 내장형 파티셔너임<br/>
각각 이산형과 연속형 값을 다룰 때 사용함<br/>
두 파티셔너는 구조적 API와 RDD 모두 사용할 수 있음<br/>

In [49]:
import org.apache.spark.HashPartitioner

rdd.map(r => r(6)).take(30).foreach(println)

17850
17850
17850
17850
17850
17850
17850
17850
17850
13047
13047
13047
13047
13047
13047
13047
13047
13047
13047
13047
13047
13047
13047
13047
13047
13047
12583
12583
12583
12583


import org.apache.spark.HashPartitioner


In [50]:
val keyedRDD = rdd.keyBy(row => row(6).asInstanceOf[Int].toDouble)

keyedRDD.partitionBy(new HashPartitioner(10)).take(10)

keyedRDD: org.apache.spark.rdd.RDD[(Double, org.apache.spark.sql.Row)] = MapPartitionsRDD[110] at keyBy at <console>:38
res43: Array[(Double, org.apache.spark.sql.Row)] = Array((15100.0,[536374,21258,VICTORIAN SEWING BOX LARGE,32,12/1/2010 9:09,10.95,15100,United Kingdom]), (16250.0,[536388,21754,HOME BUILDING BLOCK WORD,3,12/1/2010 9:59,5.95,16250,United Kingdom]), (16250.0,[536388,21755,LOVE BUILDING BLOCK WORD,3,12/1/2010 9:59,5.95,16250,United Kingdom]), (16250.0,[536388,21523,DOORMAT FANCY FONT HOME SWEET HOME,2,12/1/2010 9:59,7.95,16250,United Kingdom]), (16250.0,[536388,21363,HOME SMALL WOOD LETTERS,3,12/1/2010 9:59,4.95,16250,United Kingdom]), (16250.0,[536388,21411,GINGHAM HEART  DOORSTOP RED,3,12/1/2010 9:59,4.25,16250,United Kingdom]), (16250.0,[536388,22318,FIVE HEART HANGIN...


HashPartitioner와 RangePartitioner는 유용하지만 매우 기초적인 기능을 제공함<br/>
매우 큰 데이터나 심각하게 치우친 키를 다뤄야 한다면 고급 파티셔닝 기능을 사용해야 함<br/>
병렬성을 개선하고 실행 과정에서 OutOfMemoryError를 방지할 수 있도록 키를 최대한 분할해야 함<br/>

키가 특정 형태를 띠는 경우에는 키를 분할해야 함<br/>
예를 들어 데이터셋에 항상 분석 작업을 어렵게 만드는 두 명의 고객 정보가 있다면 다른 고객의 정보와 두 고객의 정보를 분리해야 함<br/>
물론 다른 고객의 정보를 하나의 그룹으로 묶어서 처리할 수도 있음<br/>
하지만 두 고객의 정보와 관련된 데이터가 너무 많아 치우침이 심하게 발생한다면 나누어 처리해야 함<br/>

In [51]:
import org.apache.spark.Partitioner

class DomainPartitioner extends Partitioner {
    def numPartitions = 3
    def getPartition(key: Any): Int = {
        val customerId = key.asInstanceOf[Double].toInt
        
        if(customerId == 17850.0 || customerId == 12583.0) {
            return 0
        }
        else {
            return new java.util.Random().nextInt(2) + 1
        }
    }
}

import org.apache.spark.Partitioner
defined class DomainPartitioner


In [64]:
keyedRDD
    .partitionBy(new DomainPartitioner).map(_._1).glom().map(_.toSet.toSeq.length)
    .take(5)

res56: Array[Int] = Array(2, 4305, 4307)


위 코드 예제를 실행하면 각 파티션 수를 확인할 수 있음<br/>
데이터를 임의로 분산하였으므로 마지막 두 숫자가 다를 수 있음<br/>
하지만 동일한 원칙이 적용됨<br/>

사용자 정의 키 분배 로직은 RDD 수준에서만 사용할 수 있음<br/>
사용자 정의 키 분배 로직은 임의의 로직을 사용해 물리적인 방식으로 클러스터에 데이터를 분배하는 강력한 방법임<br/>

# 13.6 사용자 정의 직렬화

마지막 고급 주제는 **Kyro 직렬화** 관련 내용임<br/>
병렬화 대상인 모든 객체나 함수는 직렬화할 수 있어야 함<br/>

In [66]:
class SomeClass extends Serializable {
    var someValue = 0
    def setSomeValue(i: Int) = {
        someValue = i
        this
    }
}

sc.parallelize(1 to 10).map(num => new SomeClass().setSomeValue(num))

defined class SomeClass
res58: org.apache.spark.rdd.RDD[SomeClass] = MapPartitionsRDD[165] at map at <console>:24


*기본 직렬화 기능은 매우 느릴 수 있음<br/>
스파크는 Kyro 라이브러리(버전 2)를 사용해 더 빠르게 직렬화할 수 있음<br/>
Kyro는 자바 직렬화보다 약 10배 이상 성능이 좋으며 더 간결함<br/>
하지만 모든 직렬화 유형을 지원하지는 않음*<br/>
그리고 최상의 성능을 얻기 위해 프로그램에서 사용할 클래스를 사전에 등록해야 함<br/>

SparkConf를 사용해 job을 초기화하는 시점에서 spark.serializer 속성값을 org.apache.spark.serializer.KyroSerializer로 설정해 Kyro를 사용할 수 있음<br/>
Kyro는 4부에서 자세히 알아보겠음<br/>
spark.serializer 설정으로 워커 노드 간 데이터 셔플링과 RDD를 직렬화해 디스크에 저장하는 용도로 사용할 serializer를 지정할 수 있음<br/>
Kyro가 기본값이 아닌 유일한 이유는 사용자가 직접 클래스를 등록해야 하기 때문임<br/>
그리고 네트워크에 민감한 애플리케이션에서 사용할 것을 권장함<br/>
스파크 2.0.0 버전부터는 단순 데이터 타입, 단순 데이터 타입의 배열, 문자열 데이터 타입의 RDD를 셔플링하면 내부적으로 Kyro 시리얼라이저를 사용함<br/>

스파크는 Twitter chill 라이브러리의 AllScalaRegistrar에서 다루는 핵심 스칼라 클래스를 자동으로 Kyro 시리얼라이저에 등록함<br/>

Kyro에 사용자 정의 클래스를 등록하려면 registerKyroClasses 메서드를 사용함<br/>

In [67]:
/*
val conf = new SparkConf().setMaster(...).setAppName(...)
conf.registerKyroClasses(Array(classOf[MyClass1], classOf[MyClass2]))
val sc = new SparkContext(conf)
*/

# 13.7 정리

이 장에서는 RDD의 여러 고급 주제에 대해 알아보았음<br/>
*특히 데이터를 배치할 수 있는 기능을 설명한 사용자 정의 파티셔닝(13.5.4)을 주목해야 함*<br/>
다음 장에서는 스파크의 또 다른 저수준 API의 기능인 분산형 공유 변수를 알아보겠음<br/>